In [1]:
from datasets import load_from_disk
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, TrainingArguments, Trainer, EarlyStoppingCallback, AutoTokenizer, DefaultDataCollator
from huggingface_hub import login
from dotenv import load_dotenv
import os
import wandb

In [2]:
WB_KEY = os.getenv("WB_KEY")
wandb.login(key=WB_KEY)
run = wandb.init(project="Digital Self-Replica", job_type="Training", name="Train with rank=16")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Francesco\_netrc
wandb: Currently logged in as: francescobrigante (francescobrigante_s_projects) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
#pip install -U bitsandbytes

### Setting parameters

In [4]:
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
if (HF_TOKEN == None):
    raise ValueError("HF_TOKEN is not set")
login(token=HF_TOKEN)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
# 4 bit quantization
# could be further increased to 8b for more precision
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# LoRA configuration for Qwen model architecture
lora_config = LoraConfig(
    r=16,                       #rank of the added low-rank matrices
    lora_alpha=32,              #generally 2*r
    target_modules=[            #modules where LoRA is applied
        "q_proj",               # query, key, value, output projection layers in the self-attention
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",            # gate, up, down are part of the FFNN in the model
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [6]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# preparing model for LoRA
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)


# TRY  AND BATCH 8x8

# training arguments
training_args = TrainingArguments(
    output_dir="./francesco_lora",
    num_train_epochs=4,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,      # effective batch size = per_device_train_batch_size * gradinet_accumulation_steps
    #per_device_eval_batch_size=4,
    #eval_accumulation_steps=6,
    #warmup_steps=5,
    warmup_ratio=0.1,
    learning_rate=2e-4,                # Slightly lower for distilled model
    optim="paged_adamw_8bit",         # 8bit optimizer <- ADDED
    lr_scheduler_type="cosine",       # cosine learning rate scheduler <- ADDED
    weight_decay=0.01,
    fp16=True,
    logging_steps=25,
    #eval_strategy="no",
    eval_strategy="steps",
    #per_device_eval_batch_size=16,
    #eval_accumulation_steps=2,
    eval_steps=100,
    save_steps=100,
    save_strategy="steps",
    load_best_model_at_end=True,
    #save_total_limit=1,
    metric_for_best_model="loss",
    greater_is_better=False,            #lower loss is better
    gradient_checkpointing=False,
    max_grad_norm=0.5,
    disable_tqdm=False,
    report_to=["wandb"],                                # Enable W&B logging
    #report_to=["none"],                                # Enable W&B logging
    label_names=["labels"]
)

In [8]:
# training_args = TrainingArguments(
#     output_dir="./francesco_lora",
#     num_train_epochs=3,
#     per_device_train_batch_size=6,
#     gradient_accumulation_steps=4,      # effective batch size = per_device_train_batch_size * gradinet_accumulation_steps
#     #per_device_eval_batch_size=4,
#     #eval_accumulation_steps=6,
#     warmup_steps=5,
#     warmup_ratio=0.1,
#     learning_rate=2e-4,                # Slightly lower for distilled model
#     optim="paged_adamw_8bit",         # 8bit optimizer <- ADDED
#     lr_scheduler_type="cosine",       # cosine learning rate scheduler <- ADDED
#     weight_decay=0.01,
#     fp16=True,
#     logging_steps=5,
#     #eval_strategy="no",
#     eval_strategy="steps",
#     per_device_eval_batch_size=16,
#     eval_accumulation_steps=2,
#     eval_steps=5,
#     save_steps=50,
#     save_strategy="steps",
#     load_best_model_at_end=True,
#     #save_total_limit=1,
#     metric_for_best_model="loss",
#     greater_is_better=False,            #lower loss is better
#     gradient_checkpointing=False,
#     max_grad_norm=0.5,
#     disable_tqdm=False,
#     #report_to=["wandb"],                                # Enable W&B logging
#     report_to=["none"],                                # Enable W&B logging
#     label_names=["labels"]
# )

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

data_collator = DefaultDataCollator()

# loading datasets
tokenized_train = load_from_disk('datasets/tokenized_train')
tokenized_val = load_from_disk('datasets/tokenized_val')
#tokenized_test = load_from_disk('datasets/tokenized_test')

# print
print(f"Training examples: {len(tokenized_train)}")
print(f"Validation examples: {len(tokenized_val)}")
#print(f"Test examples: {len(tokenized_test)}")

print("\nOne training example:")
print(tokenized_train[1000])

Training examples: 8720
Validation examples: 1090

One training example:
{'input_ids': [151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 

In [10]:
#IF ON COLAB
# from google.colab import drive

# drive.mount('/content/drive')
# drive_base_path = '/content/drive/My Drive/datasets'

# tokenized_train = load_from_disk(os.path.join(drive_base_path, 'tokenized_train'))
# tokenized_val = load_from_disk(os.path.join(drive_base_path, 'tokenized_val'))
# tokenized_test = load_from_disk(os.path.join(drive_base_path, 'tokenized_test'))

# print("Datasets loaded successfully from Google Drive!")
# print(f"Training examples: {len(tokenized_train)}")
# print(f"Validation examples: {len(tokenized_val)}")
# print(f"Test examples: {len(tokenized_test)}")

# print("\nOne training example:")
# print(tokenized_train[8000])

### Actual training

In [11]:
# print trainable parameters
model.print_trainable_parameters()

# training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

# add early stopping
early_stopping = EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)
trainer.add_callback(early_stopping)

trainable params: 40,370,176 || all params: 7,655,986,688 || trainable%: 0.5273


In [12]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/2180 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\Francesco\anaconda3\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Francesco\anaconda3\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\Francesco\anaconda3\lib\site-packages\torch\utils\c

{'loss': 8.2295, 'grad_norm': 6.230851650238037, 'learning_rate': 2.2935779816513765e-05, 'epoch': 0.05}
{'loss': 5.0601, 'grad_norm': 2.8211240768432617, 'learning_rate': 4.587155963302753e-05, 'epoch': 0.09}
{'loss': 4.2819, 'grad_norm': 3.2490782737731934, 'learning_rate': 6.880733944954129e-05, 'epoch': 0.14}
{'loss': 4.0453, 'grad_norm': 4.860716819763184, 'learning_rate': 9.174311926605506e-05, 'epoch': 0.18}


  0%|          | 0/137 [00:00<?, ?it/s]

...

  0%|          | 0/137 [00:00<?, ?it/s]

{'eval_loss': 3.4414234161376953, 'eval_runtime': 750.5605, 'eval_samples_per_second': 1.452, 'eval_steps_per_second': 0.183, 'epoch': 2.39}


c:\Users\Francesco\anaconda3\lib\site-packages\peft\utils\other.py:689: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8279a6d0-b542-44fe-8f46-e34d56e7b0f1)') - silently ignoring the lookup for the file config.json in deepseek-ai/DeepSeek-R1-Distill-Qwen-7B.
  warnings.warn(
c:\Users\Francesco\anaconda3\lib\site-packages\peft\utils\save_and_load.py:243: UserWarning: Could not find a config file in deepseek-ai/DeepSeek-R1-Distill-Qwen-7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'train_runtime': 47609.3077, 'train_samples_per_second': 0.733, 'train_steps_per_second': 0.046, 'train_loss': 3.186554688673753, 'epoch': 2.39}


TrainOutput(global_step=1300, training_loss=3.186554688673753, metrics={'train_runtime': 47609.3077, 'train_samples_per_second': 0.733, 'train_steps_per_second': 0.046, 'total_flos': 2.271875753312256e+17, 'train_loss': 3.186554688673753, 'epoch': 2.385321100917431})

In [13]:
#trainer.train(resume_from_checkpoint="./francesco_lora/checkpoint-50")